a

https://python.langchain.com/docs/tutorials/agents/

In [ ]:
%pip install -U langchain-community langgraph langchain-anthropic tavily-python langgraph-checkpoint-sqlite --quiet

In [27]:
pip install -U langchain-google-vertexai --quiet

Note: you may need to restart the kernel to use updated packages.


In [51]:
import os
from dotenv import load_dotenv
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain.chat_models import init_chat_model
from langchain_core.messages import HumanMessage
from langgraph.prebuilt import create_react_agent
from langgraph.checkpoint.memory import MemorySaver

In [47]:
load_dotenv()

True

In [48]:
assert "LANGSMITH_TRACING" in os.environ, "Please set the LANGSMITH_TRACING environment variable."
assert "LANGSMITH_API_KEY" in os.environ, "Please set the LANGSMITH_API_KEY environment variable."
assert "TAVILY_API_KEY" in os.environ, "Please set the TAVILY_API_KEY environment variable."

In [25]:
search = TavilySearchResults(max_results=2)
search_results = search.invoke("what is the weather in SF")
print(search_results)
tools = [search]

[{'title': 'Weather in San Francisco', 'url': 'https://www.weatherapi.com/', 'content': "{'location': {'name': 'San Francisco', 'region': 'California', 'country': 'United States of America', 'lat': 37.775, 'lon': -122.4183, 'tz_id': 'America/Los_Angeles', 'localtime_epoch': 1741603417, 'localtime': '2025-03-10 03:43'}, 'current': {'last_updated_epoch': 1741602600, 'last_updated': '2025-03-10 03:30', 'temp_c': 6.7, 'temp_f': 44.1, 'is_day': 0, 'condition': {'text': 'Partly cloudy', 'icon': '//cdn.weatherapi.com/weather/64x64/night/116.png', 'code': 1003}, 'wind_mph': 3.6, 'wind_kph': 5.8, 'wind_degree': 352, 'wind_dir': 'N', 'pressure_mb': 1016.0, 'pressure_in': 30.01, 'precip_mm': 0.0, 'precip_in': 0.0, 'humidity': 79, 'cloud': 50, 'feelslike_c': 5.8, 'feelslike_f': 42.4, 'windchill_c': 8.1, 'windchill_f': 46.7, 'heatindex_c': 8.1, 'heatindex_f': 46.5, 'dewpoint_c': 7.7, 'dewpoint_f': 45.8, 'vis_km': 16.0, 'vis_miles': 9.0, 'uv': 0.0, 'gust_mph': 5.8, 'gust_kph': 9.4}}", 'score': 0.883

In [30]:
model = init_chat_model("gemini-2.0-flash-001", model_provider="google_vertexai")

In [33]:
response = model.invoke([HumanMessage(content="hi!")])
response.content

'Hi there! How can I help you today?\n'

In [34]:
model_with_tools = model.bind_tools(tools)

In [35]:
response = model_with_tools.invoke([HumanMessage(content="Hi!")])

print(f"ContentString: {response.content}")
print(f"ToolCalls: {response.tool_calls}")

ContentString: Hi there! How can I help you today?

ToolCalls: []


In [36]:
response = model_with_tools.invoke([HumanMessage(content="What's the weather in SF?")])

print(f"ContentString: {response.content}")
print(f"ToolCalls: {response.tool_calls}")

ContentString: 
ToolCalls: [{'name': 'tavily_search_results_json', 'args': {'query': 'weather in San Francisco'}, 'id': 'f0e9c69b-b5e0-4004-bcb2-9463f4bd43d5', 'type': 'tool_call'}]


In [38]:
agent_executor = create_react_agent(model, tools)

In [39]:
response = agent_executor.invoke({"messages": [HumanMessage(content="hi!")]})

response["messages"]

[HumanMessage(content='hi!', additional_kwargs={}, response_metadata={}, id='5cd03ad0-0d75-4d63-8579-7a8c3138d3fa'),
 AIMessage(content='Hi there! How can I help you today?\n', additional_kwargs={}, response_metadata={'is_blocked': False, 'safety_ratings': [], 'usage_metadata': {'prompt_token_count': 49, 'candidates_token_count': 11, 'total_token_count': 60, 'prompt_tokens_details': [{'modality': 1, 'token_count': 49}], 'candidates_tokens_details': [{'modality': 1, 'token_count': 11}], 'cached_content_token_count': 0, 'cache_tokens_details': []}, 'finish_reason': 'STOP', 'avg_logprobs': -0.005166638981212269}, id='run-0903bb94-9a17-41c9-9e7e-445bc8327639-0', usage_metadata={'input_tokens': 49, 'output_tokens': 11, 'total_tokens': 60})]

In [44]:
response["messages"][0]

HumanMessage(content='hi!', additional_kwargs={}, response_metadata={}, id='5cd03ad0-0d75-4d63-8579-7a8c3138d3fa')

In [45]:
response["messages"][1]

AIMessage(content='Hi there! How can I help you today?\n', additional_kwargs={}, response_metadata={'is_blocked': False, 'safety_ratings': [], 'usage_metadata': {'prompt_token_count': 49, 'candidates_token_count': 11, 'total_token_count': 60, 'prompt_tokens_details': [{'modality': 1, 'token_count': 49}], 'candidates_tokens_details': [{'modality': 1, 'token_count': 11}], 'cached_content_token_count': 0, 'cache_tokens_details': []}, 'finish_reason': 'STOP', 'avg_logprobs': -0.005166638981212269}, id='run-0903bb94-9a17-41c9-9e7e-445bc8327639-0', usage_metadata={'input_tokens': 49, 'output_tokens': 11, 'total_tokens': 60})

In [46]:
response = agent_executor.invoke(
    {"messages": [HumanMessage(content="whats the weather in sf?")]}
)
response["messages"]

[HumanMessage(content='whats the weather in sf?', additional_kwargs={}, response_metadata={}, id='a7e4118f-8fc4-4f22-bfc9-8a2d7c717c27'),
 AIMessage(content='', additional_kwargs={'function_call': {'name': 'tavily_search_results_json', 'arguments': '{"query": "weather in San Francisco"}'}}, response_metadata={'is_blocked': False, 'safety_ratings': [], 'usage_metadata': {'prompt_token_count': 53, 'candidates_token_count': 13, 'total_token_count': 66, 'prompt_tokens_details': [{'modality': 1, 'token_count': 53}], 'candidates_tokens_details': [{'modality': 1, 'token_count': 13}], 'cached_content_token_count': 0, 'cache_tokens_details': []}, 'finish_reason': 'STOP', 'avg_logprobs': -0.00017368059175518842}, id='run-e48eaac3-3054-44a0-969e-4065181bcd60-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'weather in San Francisco'}, 'id': '4a1f8fbb-20a7-4f1e-9cf0-a99575b3a229', 'type': 'tool_call'}], usage_metadata={'input_tokens': 53, 'output_tokens': 13, 'total_tokens'

In [49]:
for step in agent_executor.stream(
    {"messages": [HumanMessage(content="whats the weather in sf?")]},
    stream_mode="values",
):
    step["messages"][-1].pretty_print()

================================ Human Message =================================

whats the weather in sf?
================================== Ai Message ==================================
Tool Calls:
  tavily_search_results_json (a54ec707-e375-41dd-a37b-4babe812e9cd)
 Call ID: a54ec707-e375-41dd-a37b-4babe812e9cd
  Args:
    query: weather in San Francisco
================================= Tool Message =================================
Name: tavily_search_results_json

[{"title": "Weather in San Francisco", "url": "https://www.weatherapi.com/", "content": "{'location': {'name': 'San Francisco', 'region': 'California', 'country': 'United States of America', 'lat': 37.775, 'lon': -122.4183, 'tz_id': 'America/Los_Angeles', 'localtime_epoch': 1741629281, 'localtime': '2025-03-10 10:54'}, 'current': {'last_updated_epoch': 1741628700, 'last_updated': '2025-03-10 10:45', 'temp_c': 10.6, 'temp_f': 51.1, 'is_day': 1, 'condition': {'text': 'Partly cloudy', 'icon': '//cdn.weatherapi.com/weather/6

In [50]:
for step, metadata in agent_executor.stream(
    {"messages": [HumanMessage(content="whats the weather in sf?")]},
    stream_mode="messages",
):
    if metadata["langgraph_node"] == "agent" and (text := step.text()):
        print(text, end="|")

The| weather in San| Francisco is partly cloudy with a temperature of 51.1°F (10|.6°C). The wind is coming from the NNE at 3|.8 mph.|

In [52]:
memory = MemorySaver()

In [53]:
agent_executor = create_react_agent(model, tools, checkpointer=memory)

config = {"configurable": {"thread_id": "abc123"}}

In [54]:
for chunk in agent_executor.stream(
    {"messages": [HumanMessage(content="hi im bob!")]}, config
):
    print(chunk)
    print("----")

{'agent': {'messages': [AIMessage(content='Hi Bob! How can I help you today?\n', additional_kwargs={}, response_metadata={'is_blocked': False, 'safety_ratings': [], 'usage_metadata': {'prompt_token_count': 51, 'candidates_token_count': 11, 'total_token_count': 62, 'prompt_tokens_details': [{'modality': 1, 'token_count': 51}], 'candidates_tokens_details': [{'modality': 1, 'token_count': 11}], 'cached_content_token_count': 0, 'cache_tokens_details': []}, 'finish_reason': 'STOP', 'avg_logprobs': -0.06646347045898438}, id='run-e89d9cc7-203c-4526-9718-ddac055e4253-0', usage_metadata={'input_tokens': 51, 'output_tokens': 11, 'total_tokens': 62})]}}
----
